# GRT Integration test

In this notebook we will perform package integration tests

In [125]:
from gravray import *
from gravray.util import *
from gravray.sampling import *
from gravray.spice import *
from gravray.orbit import *
from gravray.stats import *

from tqdm import tqdm
import pandas as pd

Spice.loadKernels()

In [93]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Test 1: Earth

We will restrict first to Earth

In [94]:
body="EARTH"
earth=Body(body)

### Test 1.1: Single site, single time, multiple rays

Let's use a well known impact

In [96]:
lon=61.1*Angle.Deg #deg
lat=54.8*Angle.Deg #deg
alt=23.3*Const.km #m
chelyabinsk=Location(earth,lon,lat,alt)
tdb=Spice.str2t("02/15/2013 03:20:34 UTC")
ray=GrtRay(chelyabinsk,101.1*Angle.Deg,15.9*Angle.Deg,-18.6*Const.km)
ray.updateRay(tdb)
ray.propagateRay(tdb)
raydf=ray.packRay()
Util.transformElements(ray.terminal.elements,[1/Const.au,Angle.Rad])

array([  0.73858152,   0.54966804,   4.0415789 , 326.57255584,
       106.8634198 ,  21.32354715])

Now we generate a list of sample directions distributed on a hemisphere following the Fibonacci spiral. We will use a set of regularly spaced impact speeds as in the original GRT:

In [134]:
directions=Sample(1000)
directions.genUnitHemisphere()
speeds=np.linspace(11.2,72.0,50)*Const.km/Const.s

In [140]:
et=elTime(0)
nfailed=0
nhyper=0
nnormal=0
raysdf=pd.DataFrame()
for i,direction in enumerate(tqdm(directions.pp)):
    h=np.pi/2-direction[1]
    A=direction[2]
    for j,v in enumerate(speeds[0:]):
        ray=GrtRay(chelyabinsk,A,h,-v)
        ray.updateRay(tdb)
        try:
            ray.propagateRay(tdb)
            if ray.terminal.elements[1]>1:
                nhyper+=1
            else:
                nnormal+=1
        except AssertionError as e:
            nfailed+=1
            continue
        raydf=ray.packRay()
        raydf["detJ"]=ray.calcJacobianDeterminant()
        raysdf=pd.concat((raysdf,raydf))
raysdf=raysdf.reset_index()
et=elTime()
nnormal,nfailed,nhyper

  0%|          | 1/1000 [00:01<26:24,  1.59s/it]


KeyboardInterrupt: 

In [148]:
#raysdf=pd.read_csv("rays.csv") #In case of accident
#raysdf.to_csv("rays.csv",index=False)
#np.savetxt("rays.dat",raysdf.values)